# Notebook: E-handelsplattformen (Grupp 1: Daniel, Hamed, Martin W)

Vi är ett team som analyserar försäljningsdata från en påhittad e-handelsplattform.  
Vi ska ge ledningen ett snabbt beslutsunderlag inför nästa kampanjperiod.

Datasetet innehåller:
- `order_id`
- `date`
- `city`
- `category`
- `price`
- `units`
- `revenue`

In [ ]:
# För att få relative path utifrån notebookens workspace mapp.
import sys, os 
sys.path.append(os.path.abspath(".."))

In [ ]:
# Läs in data
from src.io_utils import load_csv
df = load_csv("../data/ecommerce_sales.csv")
# Kolla 2 rader
df.head(2)

In [ ]:
# Importera funktioner 
from src.metrics import total_revenue, total_units, aov, revenue_by_category, revenue_by_city, top_categories, median_revenue, standard_deviation, revenue_over_time
import pandas as pd

---

Vi ska svara följande affärsfrågor:
1. Vad säljer? – Vilka kategorier driver mest intäkt?  
2. Var säljer vi? – Vilka städer står för störst intäkt?  
3. När säljer vi? – Finns säsongsmönster i försäljningen?  
4. Hur ser en typisk order ut? – AOV (Average Order Value).  
5. Topp-3 kategorier efter intäkt.  
6. Eventuella avvikelser.  

In [ ]:
# Visa metrics
print("Total Revenue:", total_revenue(df))
print("Total Units Sold:", total_units(df))
print("Average Order Value (AOV):", aov(df))

print("\nRevenue by Category:\n", revenue_by_category(df))
print("\nRevenue by City:\n", revenue_by_city(df))
print("\nTop Categories:\n", top_categories(df))
print("\nMedian Revenue:\n", median_revenue(df))
print("\nStandard deviation:\n", standard_deviation(df))

In [ ]:
# Fråga 3 (vecko-snitt)
rev = pd.DataFrame(revenue_over_time(df))
rev.head(2)

rev["row_nr"] = range(len(rev))             # gör radnummer kolumn
rev["weekly"] = (rev["row_nr"] // 7) + 1    # gör veckonummer kolumn
rev.head(2)

weekly_avg = rev.groupby("weekly")['revenue'].mean() # aggregera värden
weekly_avg.plot(x="date",y="revenue",title="Försäljning per vecka") # enkel plot

In [ ]:
# Fråga 3 (månads-snitt)
rev["monthly"] = (rev["row_nr"] // 30) + 1 # gör månadnummer kolumn
monthly_avg = rev.groupby("monthly")["revenue"].mean() # aggregera värden
monthly_avg.plot(x="date",y="revenue",title="Försäljning per månad") # enkel plot

In [ ]:
q1 = df["revenue"].quantile(0.25)
q3 = df["revenue"].quantile(0.75)
iqr = q3 - q1
high_bound = q3 + 1.5 * iqr
lower_bound = q1 - 1.5 * iqr
temp = df[df['revenue'] > high_bound]
len(temp)

In [ ]:
# Anekdot: vi kan titta på z-score istället för IQR outliers.
z = (df['revenue'] - df['revenue'].mean()) / df['revenue'].std()
z

In [ ]:
# Vi testar nu dataklassen vi skapat i ecommerce.py
from src.ecommerce import Analyzer

TEMP = Analyzer(df)
TEMP.compute_all_calculations()
TEMP.make_the_plots()
print(TEMP.summary())

---

In [ ]:
# Visualizations
from src.viz import plot_revenue_by_category, plot_sales_over_time

fig1, ax1 = plot_revenue_by_category(df)
fig2, ax2 = plot_sales_over_time(df)